сайт соревнования Semeval-2021 Task 2: [Multilingual and Cross-lingual Word-in-Context Disambiguation](https://competitions.codalab.org/competitions/27054)

In [1]:
!git clone -q  https://github.com/DanilDmitriev1999/ABBYY_HW_NLP

In [2]:
!pip install --upgrade -r /content/ABBYY_HW_NLP/WiC/requirements.txt

     |████████████████████████████████| 7.0MB 4.1MB/s 
     |████████████████████████████████| 12.8MB 245kB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 696kB 30.3MB/s 
     |████████████████████████████████| 2.3MB 28.3MB/s 
     |████████████████████████████████| 1.2MB 20.4MB/s 
     |████████████████████████████████| 22.3MB 1.6MB/s 
     |████████████████████████████████| 266kB 46.2MB/s 
     |████████████████████████████████| 829kB 30.5MB/s 
     |████████████████████████████████| 112kB 42.3MB/s 
     |████████████████████████████████| 276kB 44.9MB/s 
     |████████████████████████████████| 3.3MB 30.5MB/s 
     |████████████████████████████████| 901kB 29.5MB/s 
     |████████████████████████████████| 522kB 45.6MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 1.3MB 29.3MB/s 
     |████████████████████████████████| 143kB 44.3MB/

In [3]:
!pip install -q jsonlines

In [4]:
import warnings
warnings.filterwarnings('ignore')
import comet_ml

import os
import torch
import random
import numpy as np
import jsonlines
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
from torch.utils.data import Dataset, RandomSampler, DataLoader
from torch import nn
from torch.nn import functional as F

from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.metrics import Accuracy
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import EarlyStopping  
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup, AutoModel
from transformers import BertTokenizerFast


comet_ml is installed but `COMET_API_KEY` is not set.


In [5]:
from ABBYY_HW_NLP.WiC.DataModule.custom_dataset import *
from ABBYY_HW_NLP.WiC.DataModule.utils import *

In [7]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [20]:
df.head()

,idx,word,sentence1,sentence2,start1,end1,start2,end2,label,gold_sense1,gold_sense2
0,0,двор,В нашей деревне осталось от силы двадцать дворов,"Солнце стояло уже высоко, когда справа от доро...",42,49,69,76,True,1,1
1,1,доклад,Табличка на дверях: «Без доклада не входить»,Вчера отбыл в столицу первый секретарь обкома ...,25,33,97,104,False,4,2
2,2,засада,"У нас вообще […] засада с героями, способными ...",Там в воскресенье все магазины закрыты – вот в...,17,24,50,57,True,2,2
3,3,доля,"Он не успел сказать и десятой доли того, что с...",Болезнь ее была странного свойства – […] что-т...,30,35,70,77,False,2,3
4,4,закат,"Теперь, если она не пойдет звонить мужу, успее...","Тридцать с лишним лет службы в органах, три ра...",51,58,85,92,False,1,2


In [27]:
def read_train_data(path, read_tags=False):
    with jsonlines.open(path) as reader:
        data = [row for row in reader]
    df = pd.DataFrame(data)


    if read_tags:
        df['label'] = df['label'].replace({True: 1, False: 0})

    df['lemma'] = df['word'].apply(lambda lemma: lemma.lower())
    return df

In [29]:
df_train_wic = read_train_data('/content/ABBYY_HW_NLP/WiC/data/RUSSE/train.jsonl', True)
df_dev_wic = read_train_data('/content/ABBYY_HW_NLP/WiC/data/RUSSE/val.jsonl', True)
print(df_train_wic.shape)
print(df_dev_wic.shape)
print(df_train_wic.head())
print(df_dev_wic.head())


(19845, 12)
(8505, 12)
   idx    word  ... gold_sense2   lemma
0    0    двор  ...           1    двор
1    1  доклад  ...           2  доклад
2    2  засада  ...           2  засада
3    3    доля  ...           3    доля
4    4   закат  ...           2   закат

[5 rows x 12 columns]
   idx       word  ... gold_sense2      lemma
0    0   здоровье  ...           1   здоровье
1    1     звание  ...           1     звание
2    2  заготовка  ...           3  заготовка
3    3       диск  ...           6       диск
4    4      дрянь  ...           2      дрянь

[5 rows x 12 columns]


### Разбиваем по леммам

In [31]:
global_df = pd.concat([df_train_wic, df_dev_wic], ignore_index=True)
df_train, df_val = lemma_train_test_split(global_df)

print(df_train.shape)
print(df_val.shape)


(23130, 12)
(5220, 12)


In [32]:
df_train.head()

,idx,word,sentence1,sentence2,start1,end1,start2,end2,label,gold_sense1,gold_sense2,lemma
2,2,засада,"У нас вообще […] засада с героями, способными ...",Там в воскресенье все магазины закрыты – вот в...,17,24,50,57,1,2,2,засада
3,3,доля,"Он не успел сказать и десятой доли того, что с...",Болезнь ее была странного свойства – […] что-т...,30,35,70,77,0,2,3,доля
5,5,демократия,Европейские демократии,"Демократию Брюсов презирал. История культуры, ...",12,23,0,11,1,1,1,демократия
6,6,заря,Заря новой эры,"Розовый свет на красноватых, со смуглотой, дре...",0,5,83,88,0,2,1,заря
7,7,задержка,Задержка взгляда на собеседнике,Влияние курса валют на нас сказывается не сраз...,0,9,53,63,0,1,6,задержка


In [34]:
train_labels = df_train["label"].tolist()
val_labels = df_val["label"].tolist()

train_sentences = [(s1, s2) for s1, s2 in zip(df_train["sentence1"], df_train["sentence2"])]
val_sentences = [(s1, s2) for s1, s2 in zip(df_val["sentence1"], df_val["sentence2"])]

train_word_ranges = [((int(s1), int(e1)), (int(s2), int(e2))) for s1, e1, s2, e2 in zip(df_train["start1"], df_train["end1"], df_train["start2"], df_train["end2"])]
val_word_ranges = [((int(s1), int(e1)), (int(s2), int(e2))) for s1, e1, s2, e2 in zip(df_val["start1"], df_val["end1"], df_val["start2"], df_val["end2"])]

print(train_sentences[0])
print(train_word_ranges[0])

('У нас вообще […] засада с героями, способными дотягивать в жизни до собственного творчества', 'Там в воскресенье все магазины закрыты – вот ведь засада!')
((17, 24), (50, 57))


### Готовим Dataset и Dataloader

In [35]:
INDICES_PADDING_LEN = 5
INDICES_PADDING_VALUE = 0
MAX_TOKENS = 100
BATCH_SIZE = 8
EPOCHS = 5

In [42]:
rubert_tokenizer = BertTokenizerFast.from_pretrained("DeepPavlov/rubert-base-cased")

In [43]:
train_data = BertDataset(train_sentences, train_word_ranges, train_labels, MAX_TOKENS, rubert_tokenizer)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler)

val_data = BertDataset(val_sentences, val_word_ranges, val_labels, MAX_TOKENS, rubert_tokenizer)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)


In [40]:
def batched_index_select(t, dim, inds):
    dummy = inds.unsqueeze(2).expand(inds.size(0), inds.size(1), t.size(2))
    out = t.gather(dim, dummy) # b x e x f
    return out

def get_mask(indices, embedding_size):
    mask = (indices != INDICES_PADDING_VALUE)
    mask.unsqueeze_(-1)
    mask = mask.expand(mask.shape[0], mask.shape[1], embedding_size)
    LARGE_VALUE = 2**32
    return torch.where(mask == True, 0, LARGE_VALUE)

def get_tokens_embeddings(batch, indices):
    return batched_index_select(batch, 1, indices) - get_mask(indices, batch.shape[2])


### используйте косинусную близость вместо линейных слоев для определения, имеют ли таргет-слова одно значение или разные

In [63]:
comet_logger = CometLogger(
    api_key="HWfJT3eyByVJWe4nEbi1pGosA",
    workspace="danildmitriev1999",
    project_name="3-sem",
    experiment_name="RUSSE RuBERT, 6, 7, 8, 9 layer, cos sim",
)

CometLogger will be initialized in online mode


In [64]:
class RUSSEClassifier(LightningModule):
    def __init__(self, model_path):
        super(RUSSEClassifier, self).__init__()

        self.model = AutoModel.from_pretrained(model_path, output_hidden_states=True)
        self.embedding_dim = self.model.get_input_embeddings().embedding_dim

        self.loss = nn.BCELoss()
        self.save_hyperparameters()

        self.valid_accuracy = Accuracy()
        self.test_accuracy = Accuracy()

        self.relu = nn.ReLU()

        self.cos_sim = nn.CosineSimilarity()

        self.threshold = 0.5
    
        self.n_layers = [6, 7, 8, 9]
    
    def _get_hidden_state(self, input_ids, attention_mask):
        tr_output = self.model(input_ids, attention_mask)
        pooled_output = torch.cat(tuple([tr_output['hidden_states'][i] for i in self.n_layers]), dim=-1)
        return pooled_output

    def _get_embeddings(self, input_ids, attention_mask, word_indices):
        sentence_outputs = self._get_hidden_state(input_ids, attention_mask)
        tokens_embeddings = get_tokens_embeddings(sentence_outputs, word_indices)
        word_embedding = torch.max(tokens_embeddings, 1)[0]
        return word_embedding

    def forward(self, input_ids, attention_mask, word_indices):
        first_embeddings = self._get_embeddings(input_ids[0], attention_mask[0], word_indices[0])
        second_embeddings = self._get_embeddings(input_ids[1], attention_mask[1], word_indices[1])

        outputs = self.cos_sim(first_embeddings, second_embeddings)
        outputs = self.relu(outputs)
        return outputs    

    def training_step(self, batch, _):
        inputs, attn, word_indices, labels = batch
        outputs = self(inputs, attn, word_indices)
        return self.loss(outputs, labels)

    def validation_step(self, batch, _):
        inputs, attn, word_indices, labels = batch
        outputs = self(inputs, attn, word_indices)

        logits = (outputs > self.threshold).float()

        self.valid_accuracy.update(logits, labels.int())
        self.log("RUSSE: val_acc", self.valid_accuracy)

        loss = self.loss(outputs, labels)
        self.log("val_loss", loss, prog_bar=True)

    def validation_epoch_end(self, _):
        self.log("RUSSE: val_acc_epoch", self.valid_accuracy.compute(), prog_bar=True)

    def test_step(self, batch, _):
        inputs, attn, word_indices, labels = batch
        outputs = self(inputs, attn, word_indices)
        self.predictions_proba.append(outputs)

        logits = (outputs > self.threshold).float()

        self.test_accuracy.update(logits, labels.int())
        self.log("test_acc", self.test_accuracy)

        loss = self.loss(outputs, labels)
        self.log("test_loss", loss)
    
    def test_epoch_end(self, _):
        self.log("test_acc_epoch", self.test_accuracy.compute())

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=1e-6)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=TOTAL_STEPS)
        return [optimizer], [scheduler]

In [65]:
TOTAL_STEPS = (len(train_data) // BATCH_SIZE + 1) * EPOCHS

# bert_model = BertModel.from_pretrained("bert-base-cased").to('cuda')
clf = RUSSEClassifier("DeepPavlov/rubert-base-cased")

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=2,
    verbose=True,
    mode="min" 
)

trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=10,
    max_epochs=EPOCHS,
    callbacks=[early_stop_callback],
    logger=[comet_logger],
    val_check_interval=0.5)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [66]:
trainer.fit(clf, train_loader, val_loader)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/danildmitriev1999/3-sem/e236047487124553959c30c9b0ee60f7


  | Name           | Type             | Params
----------------------------------------------------
0 | model          | BertModel        | 177 M 
1 | loss           | BCELoss          | 0     
2 | valid_accuracy | Accuracy         | 0     
3 | test_accuracy  | Accuracy         | 0     
4 | relu           | ReLU             | 0     
5 | cos_sim        | CosineSimilarity | 0     
----------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/danildmitriev1999/3-sem/e236047487124553959c30c9b0ee60f7
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     RUSSE: val_acc [10]       : (0.48524904251098633, 0.6457854509353638)
COMET INFO:     RUSSE: val_acc_epoch [10] : (0.48524904251098633, 0.6457854509353638)
COMET INFO:     loss [1450]               : (0.03077528439462185, 0.10028435289859772)
COMET INFO:     val_loss [10]             : (0.6388466954231262, 0.7265263795852661)
COMET INFO:   Others:
COMET INFO:     Name : RUSSE RuBERT, 6, 7, 8, 9 layer, cos sim
COMET INFO:   Parameters:
COMET INFO:     model_path : DeepPavlov/rubert-base-cased
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
CO

1

In [ ]:
trainer.test(clf, test_loader)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/danildmitriev1999/3-sem/17e7ded3b32349a29b7ec7ccfb82609b




--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.6880000233650208,
 'test_acc_epoch': 0.6880000233650208,
 'test_loss': 0.5656610131263733}
--------------------------------------------------------------------------------


[{'test_acc': 0.6880000233650208,
  'test_acc_epoch': 0.6880000233650208,
  'test_loss': 0.5656610131263733}]